# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop). Если нет аккаунта на кеггле, скачать датасет можно [здесь](https://drive.google.com/file/d/1rLSr49zx6RPZIn7PV_LQr9KnnpPhrr0K/view?usp=sharing).

# Загрузка и предобработка данных

In [1]:
import math

import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/ml_hw/'

Загрузим данные и проведем предобраотку данных как на семинаре.

In [4]:
articles_df = pd.read_csv(path + "articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [5]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3047 entries, 1 to 3121
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   timestamp        3047 non-null   int64 
 1   eventType        3047 non-null   object
 2   contentId        3047 non-null   int64 
 3   authorPersonId   3047 non-null   int64 
 4   authorSessionId  3047 non-null   int64 
 5   authorUserAgent  669 non-null    object
 6   authorRegion     669 non-null    object
 7   authorCountry    669 non-null    object
 8   contentType      3047 non-null   object
 9   url              3047 non-null   object
 10  title            3047 non-null   object
 11  text             3047 non-null   object
 12  lang             3047 non-null   object
dtypes: int64(4), object(9)
memory usage: 333.3+ KB


In [6]:
interactions_df = pd.read_csv(path + "articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [7]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72312 entries, 0 to 72311
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    72312 non-null  int64 
 1   eventType    72312 non-null  object
 2   contentId    72312 non-null  int64 
 3   personId     72312 non-null  int64 
 4   sessionId    72312 non-null  int64 
 5   userAgent    56918 non-null  object
 6   userRegion   56907 non-null  object
 7   userCountry  56918 non-null  object
dtypes: int64(4), object(4)
memory usage: 4.4+ MB


In [8]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [9]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(
    lambda x: event_type_strength[x]
)

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [10]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [11]:
interactions_from_selected_users_df = interactions_df.loc[
    np.in1d(interactions_df.personId, users_with_enough_interactions_df)
]

In [12]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [13]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [14]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [15]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    ""
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [17]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808330 sha256=5de8eb121a7067ca5c514f591c890f5ed1eddc6dffbe83a9fd3db49b725353d9
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [20]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.data import Dataset

## Задание 1 (1.5 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [89]:
lfm_dataset = Dataset()
lfm_dataset.fit(
    users=interactions_full_df['personId'],
    items=interactions_full_df['contentId']
    )

In [90]:
lfm_dataset.interactions_shape()

(1140, 2984)

In [91]:
train_inter_tuples = (
    interactions_train_df[['personId', 'contentId', 'eventStrength']]
    .apply(tuple, axis=1)
)

test_inter_tuples = (
    interactions_test_df[['personId', 'contentId', 'eventStrength']]
    .apply(tuple, axis=1)
)

In [92]:
lfm_dataset.build_interactions(train_inter_tuples)[1].toarray()

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [93]:
# Ваш код здесь
data_train = lfm_dataset.build_interactions(train_inter_tuples)[1]
data_test = lfm_dataset.build_interactions(test_inter_tuples)[1]

## Задание 2 (0.5 балла)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [94]:
# Ваш код здесь
lfm_model_warp = LightFM(loss='warp')
lfm_model_warp.fit(data_train)

In [95]:
precision_at_k(lfm_model_warp, data_test, k=10).mean()

0.002851324

In [96]:
# метрика из задания
precision_at_k(lfm_model_warp, data_test, data_train, k=10).mean()

0.00407332

In [97]:
precision_at_k(lfm_model_warp, data_test, data_train, k=5).mean()

0.0032586558

## Задание 3 (2 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [72]:
vectorizer = TfidfVectorizer()

In [112]:
interactions_full_df['contentId'].drop_duplicates()

0        -5065077552540450930
1        -6623581327558800021
2         -793729620925729327
3         1469580151036142903
4         7270966256391553686
                 ...         
38042     4106497696154898573
38152    -8464215556093549753
38476    -8202212195240926680
38514     5518462222339671372
38602     5937899505996968869
Name: contentId, Length: 2984, dtype: object

In [191]:
corpus_df = (
    interactions_full_df[['contentId']]
    .drop_duplicates()
    .merge(articles_df[['contentId', 'text', 'lang']], on='contentId', how='left')
    .fillna('')
    .reset_index(drop=True)
)

In [192]:
corpus_df.shape

(2984, 3)

In [193]:
corpus_df.head(3)

,contentId,text,lang
0,-5065077552540450930,A AXA se manteve na liderança do ranking de ma...,pt
1,-6623581327558800021,"About a decade ago, a handful of Google's most...",en
2,-793729620925729327,"Posted by Sam Thorogood , Developer Programs E...",en


In [194]:
corpus = corpus_df['text']

In [195]:
# Ваш код здесь
feat = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., '食べ物', '食事中', '건배'], dtype=object)

In [134]:
lfm_model_warp_feat = LightFM(loss='warp')
lfm_model_warp_feat.fit(data_train, item_features=feat)

In [135]:
precision_at_k(
    lfm_model_warp_feat, data_test, data_train,
    item_features=feat, k=10
    ).mean()

0.0067209774

## Задание 4 (1.5 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [148]:
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [151]:
corpus_df['lang'].value_counts()

en    2148
pt     822
         8
la       2
es       2
ja       2
Name: lang, dtype: int64

In [150]:
stopwords_punc_lst = (
    stopwords.words('english') +
    stopwords.words('portuguese') +
    list(punctuation)
)

In [152]:
def make_word_tokens(word):
    result = word_tokenize(word)
    return result

In [196]:
new_vectorizer = TfidfVectorizer(
    lowercase=True, tokenizer=make_word_tokens, stop_words=stopwords_punc_lst
    )

In [197]:
# Ваш код здесь
feat_new = new_vectorizer.fit_transform(corpus)
new_vectorizer.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


array(["''", "'+", "'+crypto.createhash", ..., '��️', '����', '������'],
      dtype=object)

In [155]:
lfm_model_warp_feat_new = LightFM(loss='warp')
lfm_model_warp_feat_new.fit(data_train, item_features=feat_new)

In [156]:
precision_at_k(
    lfm_model_warp_feat_new, data_test, data_train,
    item_features=feat_new, k=10
    ).mean()

0.005702648

Улучшилось ли качество предсказания?

Стало хуже :(

## Задание 5 (1.5 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [181]:
import tqdm

In [198]:
# Ваш код здесь
def hyperparam_search(train, test, feat, hyperparams_dict):

    results_dict = dict()

    for idx, hyperparams in tqdm.tqdm(enumerate(hyperparams_dict)):

        model = LightFM(**hyperparams, random_state=42)
        model.fit(train, item_features=feat, epochs=10, num_threads=4)

        score = precision_at_k(
            model, test, train, item_features=feat, k=10, num_threads=4
            ).mean()

        results_dict[idx] = {
            'score': score,
            'hyperparams': hyperparams
            }

    return results_dict

In [199]:
def config_generator(no_components_range, k_range, n_range, learning_rate_range):
    hyperparams = list()
    for no_components in no_components_range:
        for k in k_range:
            for n in n_range:
                for learning_rate in learning_rate_range:
                    hyperparams.append(
                        {
                            'no_components': no_components,
                            'k': k,
                            'n': n,
                            'learning_rate': learning_rate
                            }
                    )
    return hyperparams

In [200]:
hyperparams = config_generator(
    no_components_range=[5, 10, 15, 20],
    k_range=[5, 10, 15],
    n_range=[5, 10, 15],
    learning_rate_range=[0.01, 0.05, 0.75, 0.1]
    )

len(hyperparams)

144

In [204]:
search_res = hyperparam_search(data_train, data_test, None, hyperparams)
scores_dct = ({key: value['score'] for key, value in search_res.items()})

144it [00:55,  2.61it/s]


In [206]:
# лучший скор
max_key = max(scores_dct, key=lambda k: scores_dct[k])
search_res[max_key]

{'score': 0.004989817,
 'hyperparams': {'no_components': 5, 'k': 10, 'n': 5, 'learning_rate': 0.05}}

In [209]:
hyperparams_new = config_generator(
    no_components_range=[5, 10, 15],
    k_range=[5, 10],
    n_range=[5, 10],
    learning_rate_range=[0.01, 0.05]
    )

len(hyperparams_new)

24

In [210]:
search_res_feat = hyperparam_search(data_train, data_test, feat, hyperparams_new)
scores_feat_dct = ({key: value['score'] for key, value in search_res_feat.items()})

24it [11:11, 27.97s/it]


In [214]:
# лучший скор
max_key_feat = max(scores_feat_dct, key=lambda k: scores_feat_dct[k])
search_res_feat[max_key_feat]

{'score': 0.003869654,
 'hyperparams': {'no_components': 5, 'k': 5, 'n': 5, 'learning_rate': 0.01}}

In [212]:
search_res_feat_new = hyperparam_search(data_train, data_test, feat_new, hyperparams_new)
scores_feat_new_dct = ({key: value['score'] for key, value in search_res_feat_new.items()})

24it [09:59, 25.00s/it]


In [215]:
# лучший скор
max_key_feat_new = max(scores_feat_new_dct, key=lambda k: scores_feat_new_dct[k])
search_res_feat_new[max_key_feat_new]

{'score': 0.00030549898,
 'hyperparams': {'no_components': 5, 'k': 5, 'n': 5, 'learning_rate': 0.01}}

не судьба :(  
лучшая моделька - с дефолтными параметрами, на дефолтном тексте

In [217]:
lfm_model_warp_feat.get_params()

{'loss': 'warp',
 'learning_schedule': 'adagrad',
 'no_components': 10,
 'learning_rate': 0.05,
 'k': 5,
 'n': 10,
 'rho': 0.95,
 'epsilon': 1e-06,
 'max_sampled': 10,
 'item_alpha': 0.0,
 'user_alpha': 0.0,
 'random_state': RandomState(MT19937) at 0x7F7E8FFCC140}

In [218]:
precision_at_k(
    lfm_model_warp_feat, data_test, data_train,
    item_features=feat, k=10
    ).mean()

0.0067209774

## Задание 6 (1 балл)

Реализуйте функции для вычисления следующих метрик:
* precision@k
* recall@k
* NDCG@k



In [ ]:
# Ваш код здесь

## Задание 7 (1 балл)

Вычислите значения реализованных метрик для $k=10$ для лучшей полученной модели в предыдущих шагах.

Найдите уже реализованные варианты этих метрик в библиотеках lightfm и sklearn. Сравните полученные у вас значения метрик с результатами встроенных в библиотеки метрик.

In [ ]:
# Ваш код здесь

## Задание 8 (1 балл)

Реализуйте алгоритм ALS и примените его для решения задачи ноутбука.

**ALS**

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}$$

Суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя~$u$ и товара $i$, соответственно, матрицы $P, Q$ получаются путем записывания по столбцам векторов $p_u, q_i$ соответственно.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$

Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$

аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья c постановкой задачи для ALS на explicit feedback:**

* Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.

**Оригинальная статья с ALS для implicit данных, которая стала более известной:**

* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee.


In [ ]:
# Ваш код здесь